# Preprocessing Osteocyte Videos

This notebook demonstrates video preprocessing for 2D osteocyte cell cultures in `data/raw/wildtype/` and `data/raw/mutant/`, including background correction, Fourier filtering, and edge filter optimization for a user-specified number of frames for all videos. Outputs are saved in `data/processed/{condition}/{video_name}/` and `results/figures/{condition}/{video_name}/`.

In [1]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.filters import sobel, scharr, prewitt, roberts, farid, laplace
from sklearn.linear_model import LinearRegression

# Adjust the path to the root of the project
project_root = Path.cwd().parent 
sys.path.append(str(project_root))

from src.image_utils import load_video, correct_background, apply_fourier_filter, save_image
from src.segmentation import apply_edge_filters
from src.visualization import plot_edge_filters

# Paths
data_dir = Path('../data/raw')
output_dir = Path('../data/processed')
results_dir = Path('../results/figures')
output_dir.mkdir(exist_ok=True)
results_dir.mkdir(exist_ok=True)

# Prompt user for frame limit
response = input("Enter number of frames to process (e.g., 10, or 'all' for all frames): ").strip().lower()
max_frames = 0 if response == 'all' else int(response)

# Find videos in wildtype and mutant subfolders
video_paths = []
for condition_dir in [data_dir / 'wildtype', data_dir / 'mutant']:
    if condition_dir.exists():
        video_paths.extend(condition_dir.glob('*.mp4'))

if not video_paths:
    raise FileNotFoundError('No MP4 files found in data/raw/wildtype/ or data/raw/mutant/.')

# Edge filter weights
def compute_weights(image):
    edge_filters = [sobel, scharr, prewitt, roberts, farid, laplace]
    filtered_imgs = [f(image).ravel() for f in edge_filters]
    X = np.stack(filtered_imgs, axis=1)
    y = image.ravel()
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X, y)
    return reg.coef_

# Process all videos
for video_path in video_paths:
    condition = video_path.parent.name  # 'wildtype' or 'mutant'
    print(f'Processing video: {video_path.stem} ({condition})')
    # Create condition and video-specific subfolders
    condition_output_dir = output_dir / condition
    condition_results_dir = results_dir / condition
    condition_output_dir.mkdir(exist_ok=True)
    condition_results_dir.mkdir(exist_ok=True)
    video_output_dir = condition_output_dir / video_path.stem
    video_results_dir = condition_results_dir / video_path.stem
    video_output_dir.mkdir(exist_ok=True)
    video_results_dir.mkdir(exist_ok=True)
    # Load and preprocess video
    try:
        frames = load_video(str(video_path))
    except Exception as e:
        print(f"Error loading {video_path}: {e}. Skipping.")
        continue

    # Limit frames if specified
    frames_to_process = frames if max_frames == 0 else frames[:max_frames]

    # Process frames
    for frame_idx, frame in enumerate(frames_to_process):
        print(f'Processing frame {frame_idx} of {video_path.stem} ({condition})')
        # Preprocess
        corrected = correct_background(frame)
        filtered = apply_fourier_filter(corrected)
        combined = apply_edge_filters(filtered)

        # Visualize
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(frame, cmap='gray')
        axes[0].set_title(f'Original Frame {frame_idx}')
        axes[1].imshow(filtered, cmap='gray')
        axes[1].set_title(f'Fourier Filtered Frame {frame_idx}')
        axes[2].imshow(combined, cmap='gray')
        axes[2].set_title(f'Edge-Filtered Frame {frame_idx}')
        plt.tight_layout()
        plt.savefig(video_results_dir / f'{condition}_{video_path.stem}_frame_{frame_idx:04d}_preprocessing.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Plot edge filters
        plot_edge_filters(filtered, video_results_dir / f'{condition}_{video_path.stem}_frame_{frame_idx:04d}_edge_filters.png')

        # Save
        frame_prefix = f'{condition}_{video_path.stem}_frame_{frame_idx:04d}'
        save_image(filtered, str(video_output_dir / f'{frame_prefix}_filtered.tif'))
        save_image(combined, str(video_output_dir / f'{frame_prefix}_combined.tif'))

Processing video: Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 0 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 1 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 2 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 3 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 4 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 5 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 6 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 7 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 8 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 9 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing video: Confluence_Single movie_30.03.2025_no mask_C2_3 (wildtype)
Processing frame 0 of Confluence_Sin